In [ ]:
import numpy as np 
import pandas as pd 
import math
import datetime as dt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
###
from pathlib import Path
from decimal import ROUND_HALF_UP, Decimal

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
# import optuna.integration.lightgbm as lgb
import optuna

import warnings
warnings.simplefilter('ignore')

#最大表示列数・行数の指定
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 7)

In [ ]:
def read_files(dir_name):
    base_path        = Path(f'../input/jpx-tokyo-stock-exchange-prediction/{dir_name}')
    prices           = pd.read_csv(base_path / 'stock_prices.csv')
    options          = pd.read_csv(base_path / 'options.csv')
    financials       = pd.read_csv(base_path / 'financials.csv')
    trades           = pd.read_csv(base_path / 'trades.csv')
    secondary_prices = pd.read_csv(base_path / 'secondary_stock_prices.csv')
    return prices, options, financials, trades, secondary_prices

In [ ]:
# stock list、train files(５つ), sup files(５つ)を読み込む
# train files  2017-2021/12/3
# sup files    2021/12/6-
stock_list         = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv')
train_files        = read_files('train_files')
supplemental_files = read_files('supplemental_files')

In [ ]:
def merge_data(prices, options, financials, trades, secondary_prices, stock_list):

    base_df = prices.copy()
    
    # stock_listと結合
    _stock_list = stock_list.copy()
    _stock_list.rename(columns={'Close': 'Close_x'}, inplace=True) 
    base_df = base_df.merge(_stock_list, on='SecuritiesCode', how="left") 

    # tradesと結合
    # stock_listのNewMarketSegmentと紐づくよう、tradesのSection項目を編集する
    # _trades = trades.copy()
    # _trades['NewMarketSegment'] = _trades['Section'].str.split(' \(', expand=True)[0]
    # base_df = base_df.merge(_trades, on=['Date', 'NewMarketSegment'], how="left")

    # financialsと結合
    # _financials = financials.copy()
    # _financials.rename(columns={'Date': 'Date_x', 'SecuritiesCode': 'SecuritiesCode_x'}, inplace=True)
    # base_df = base_df.merge(_financials, left_on='RowId', right_on='DateCode', how="left")
    
    return base_df

In [ ]:
# 分割統合したところを調整したcloseを計算するmethod
# 0と算出された場合は、前のCloseで置換する
def generate_adjusted_close(df):

    df = df.sort_values("Date", ascending=False)
    df["CumAdjustmentFactor"] = df["AdjustmentFactor"].cumprod() # 分割統合する割合を累積させる
    
    #↓少数点四捨五入する関数ついていたが、無視してみた
    #df["AdClose"] = (df["CumAdjustmentFactor"] * df["Close"]).map(lambda x: Decimal(x).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP))
    
    df["Close"] = df["Close"].fillna(method='bfill') #nanの部分を直前の値に置換
    df["Open"] = df["Open"].fillna(method='bfill') #nanの部分を直前の値に置換
    df["High"] = df["High"].fillna(method='bfill') #nanの部分を直前の値に置換
    df["Low"] = df["Low"].fillna(method='bfill') #nanの部分を直前の値に置換
    
    
    df["AdClose"] = df["CumAdjustmentFactor"] * df["Close"]
    df["AdOpen"] = df["CumAdjustmentFactor"] * df["Open"]
    df["AdHigh"] = df["CumAdjustmentFactor"] * df["High"]
    df["AdLow"] = df["CumAdjustmentFactor"] * df["Low"]
    
    df = df.sort_values("Date")
    
    df["AdClose"] = df["AdClose"].replace(0, np.nan)
    df["AdClose"] = df["AdClose"].fillna(method='ffill') #nanの部分を直前の値に置換
    df["AdOpen"] = df["AdOpen"].replace(0, np.nan)
    df["AdOpen"] = df["AdOpen"].fillna(method='ffill') #nanの部分を直前の値に置換
    df["AdHigh"] = df["AdHigh"].replace(0, np.nan)
    df["AdHigh"] = df["AdHigh"].fillna(method='ffill') #nanの部分を直前の値に置換
    df["AdLow"] = df["AdLow"].replace(0, np.nan)
    df["AdLow"] = df["AdLow"].fillna(method='ffill') #nanの部分を直前の値に置換
    return df

# dfに上記methodを適応
def adjust_price(price):

    price["Date"] = pd.to_datetime(price["Date"], format="%Y-%m-%d")
    price = price.sort_values(["SecuritiesCode", "Date"])
    price['SecuritiesCode'] = price['SecuritiesCode'].astype('category')
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    return price

# 全てのdfをmergeしAdCloseを算出・追加
def collector(prices, options, financials, trades, secondary_prices, stock_list):
    
    base_df = merge_data(prices, options, financials, trades, secondary_prices, stock_list)
    base_df = adjust_price(base_df)# AdCloseを追加する
    
    return base_df

In [ ]:
%%time

# trainとsuppleを統合
base_df         = collector(*train_files, stock_list)        # 2017-2021/12/3
supplemental_df = collector(*supplemental_files, stock_list) # 2021/12/6-5/27
base_df         = pd.concat([base_df, supplemental_df]).reset_index(drop=True)

# 基本となるdfの表示 
base_df

#   **ここまでデータ整理**

In [ ]:
# # 「column_name で指定された項目の、periodsで指定された期間（複数）での変化率を導出し、項目として追加する関数」を生成する関数
# # 生成された関数は、特定証券コードだけを持つデータフレームが入力されることを前提としている。
# def calc_change_rate_base(column_name, periods):
#     def func(price):
#         for period in periods:
#             price[f"{column_name}_change_rate_{period}"] = price[column_name].pct_change(period)
#         return price
#     return func


# # 「column_name で指定された項目の、periodsで指定された期間（複数）での変動の度合いを導出し、項目として追加する関数」を生成する関数
# def calc_volatility_base(column_name, periods):
#     def func(price):
#         for period in periods:
#             price[f"{column_name}_volatility_{period}"] = np.log(price[column_name]).diff().rolling(window=period, min_periods=1).std()
#         return price
#     return func

# # 「column_name で指定された項目の、periodsで指定された期間（複数）での移動平均値と現在値の比率を導出し、項目として追加する関数」を生成する関数
# # 移動平均値そのものではなく、現在値に対する比率としているのは、今回のTargetが比率であるため。
# def calc_moving_average_rate_base(column_name, periods):
#     def func(price):
#         for period in periods:
#             price[f"{column_name}_average_rate_{period}"] = price[column_name].rolling(window=period, min_periods=1).mean() / price[column_name]
#         return price
#     return func

# # 終値の変動率を生成し、項目として追加する関数。これをShift-2するとTargetになる。
# def calc_target_shift2(price):
#     price['Target_shift2'] = price['Close'].pct_change()
#     return price

# # 入力データフレームを証券コード毎にグルーピングし、引数で渡された関数を適用する関数
# # functionsには↑で定義したcalc_xxxの関数のリストが渡される想定。
# def add_columns_per_code(price, functions):
#     def func(df):
#         for f in functions:
#             df = f(df)
#         return df
#     price = price.sort_values(["SecuritiesCode", "Date"])
#     price = price.groupby("SecuritiesCode").apply(func)
#     price = price.reset_index(drop=True)
#     return price

# # 入力データフレームに特徴量を追加する関数
# # 追加する項目は、基本的にレコード内の値だけを使う想定
# def add_columns_per_day(base_df):
#     """
#     ここに特徴量を入れていく
#     """
#     base_df['body'] = (base_df['AdClose'] - base_df['AdOpen']) / base_df['AdClose']
#     base_df['whole'] = (base_df['AdHigh'] - base_df['AdLow']) / base_df['AdClose']    
#     base_df["hige_upper"] = (base_df["AdHigh"] - np.maximum(base_df["AdClose"], base_df["AdOpen"]))/ base_df["AdClose"]
#     base_df["hige_lower"] = (np.minimum(base_df["AdClose"], base_df["AdOpen"]) - base_df["AdLow"])/ base_df["AdClose"]
#     base_df["body_upper_ratio"] = base_df["hige_upper"] / np.maximum(base_df["body"],0.000001)
#     base_df["body_lower_ratio"] = base_df["hige_lower"] / np.maximum(base_df["body"],0.000001)
#     base_df["upper_lower_ratio"] = base_df["hige_lower"] / np.maximum(base_df["hige_upper"], 0.00001)
    
#     # 平均足
#     base_df['heikin_cl'] = 0.25 * (base_df['AdOpen'] + base_df['AdHigh'] + base_df['AdLow'] + base_df['AdClose'])
#     base_df['heikin_op'] = base_df['heikin_cl'].ewm(1, adjust=False).mean().shift(1)  

#     # 時系列
#     base_df["month"] = base_df["Date"].dt.month
#     base_df["day"]   = base_df["Date"].dt.day
#     base_df["dow"]   = base_df["Date"].dt.dayofweek      # 曜日0-6(月-日曜日)

#     return base_df

# # 入力データフレームに特徴量を追加する関数
# def generate_features(base_df):
#     prev_column_names = base_df.columns
    
#     """
#     ここに特徴量をどんどん作成していく
#     """
#     periods = [3, 9]
#     functions = [
#         calc_change_rate_base("AdClose", periods), 
#         calc_volatility_base("AdClose", periods), 
#         calc_moving_average_rate_base("Volume", periods), 
#         calc_target_shift2
#     ]
    
#     # 証券コード単位の特徴量（移動平均等、一定期間のレコードをインプットに生成する特徴量）を追加
#     base_df = add_columns_per_code(base_df, functions)
#     # 日単位の特徴量（レコード内の値で導出できる特徴量）を追加
#     base_df = add_columns_per_day(base_df)
    
#     # 後で特徴量を選択しやすくするため、追加した項目名のリストを生成
#     add_column_names = list(set(base_df.columns) - set(prev_column_names))
#     return base_df, add_column_names

In [ ]:
"""
特徴量再度作り直した部分
ここを編集していく。
上は元々のオリジナル
"""



def add_columns_per_day(base_df, periods):
    """
    ここに特徴量を入れていく
    """
    ## ①時系列使用しない特徴量## 
    # 足の形を特徴量に
    base_df['body'] = abs((base_df['AdClose'] - base_df['AdOpen']) / base_df['AdClose'])
    base_df['whole'] = abs((base_df['AdHigh'] - base_df['AdLow']) / base_df['AdClose'])
    base_df["hige_upper"] = (base_df["AdHigh"] - np.maximum(base_df["AdClose"], base_df["AdOpen"]))/ base_df["AdClose"]
    base_df["hige_lower"] = (np.minimum(base_df["AdClose"], base_df["AdOpen"]) - base_df["AdLow"])/ base_df["AdClose"]
    base_df["body_upper_ratio"] = base_df["hige_upper"] / np.maximum(base_df["body"],0)
    base_df["body_lower_ratio"] = base_df["hige_lower"] / np.maximum(base_df["body"],0)
    base_df["upper_lower_ratio"] = base_df["hige_lower"] / np.maximum(base_df["hige_upper"], 0)

    # 時系列
    base_df["month"] = base_df["Date"].dt.month
    base_df["day"]   = base_df["Date"].dt.day
    base_df["dow"]   = base_df["Date"].dt.dayofweek      # 曜日0-6(月-日曜日)
    
    ## ②ラグ特徴量 ##
    # 平均足
    base_df['heikin_cl'] = 0.25 * (base_df['AdOpen'] + base_df['AdHigh'] + base_df['AdLow'] + base_df['AdClose'])

    # AdClose の%change
    for i in periods:
        base_df[f"AdClose_change_rate_{i}"] = base_df.groupby("SecuritiesCode")["AdClose"].pct_change(i)
    # Volumeの移動中央値
    for i in periods:
        base_df[f'{i}d_norm_vol'] = base_df.groupby("SecuritiesCode")["Volume"].apply(lambda x: x/ x.rolling(window=i).median())  
    # logをとったAdCloseのボラ(標準偏差)
    base_df["logAdClose"] = np.log(base_df["AdClose"])
    for i in periods:
        base_df[f"AdClose_vola_{i}"] = base_df.groupby("SecuritiesCode")["logAdClose"].diff().rolling(window=i, min_periods=1).std() 
    # Volumeのボラ
    for i in periods:
        base_df[f"Volume_vola_{i}"] = base_df.groupby("SecuritiesCode")["Volume"].diff().rolling(window=i, min_periods=1).std() 
    
    
    
    return base_df


def generate_features(base_df):
    prev_column_names = base_df.columns

    periods = [3,9,25]
    
    # 特徴量を追加
    base_df = add_columns_per_day(base_df, periods)
    
    # 後で特徴量を選択しやすくするため、追加した項目名のリストを生成
    add_column_names = list(set(base_df.columns) - set(prev_column_names))
    return base_df, add_column_names

In [ ]:
# c = a_df[a_df["SecuritiesCode"]==1301]
# plt.plot(c["Date"], c["AdClose_vola_9"])

In [ ]:
# 特徴量選択
def select_features(feature_df, add_column_names):
    
    base_cols = ['RowId', 'Date', 'SecuritiesCode']
    # 数値系の特徴量
    numerical_cols = sorted(add_column_names)
    # カテゴリ系の特徴量
    categorical_cols = ['NewMarketSegment', '33SectorCode', '17SectorCode']
    # 目的変数
    label_col = ['Target']
    
    # 選んだ特徴量
    feat_cols = numerical_cols + categorical_cols

    # データフレームの項目を選択された項目だけに絞込
    feature_df = feature_df[base_cols + feat_cols + label_col]
    # カテゴリ系項目はdtypeをcategoryに変更
    feature_df[categorical_cols] = feature_df[categorical_cols].astype('category')

    return feature_df, feat_cols, label_col

# 特徴量生成と選択する
def preprocessor(base_df):
    feature_df = base_df.copy()
    
    ## 特徴量生成
    feature_df, add_column_names = generate_features(feature_df)
    
    ## 特徴量選択
    feature_df, feat_cols, label_col = select_features(feature_df, add_column_names)

    return feature_df, feat_cols, label_col

In [ ]:
feature_df, feat_cols, label_col = preprocessor(base_df)

# # 特徴量
# feat_cols

# sharp ratioを計算

In [ ]:
# 予測値の順位番号を振る関数
def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

# 日ごとに分けられているDFをランクごとに並べ替え、上200, 下200に対してweightを掛けて、平均でわりその差を取っている=sharp ratioの計算
# ここの計算方法の意味ががわかっていない！！
def calc_return_per_day(df):
    weights = np.linspace(start=2, stop=1, num=200)
    long  = (df.sort_values('Rank', ascending=True )['Target'][:200] * weights).sum() / weights.mean()
    short = (df.sort_values('Rank', ascending=False)['Target'][:200] * weights).sum() / weights.mean()
    return long - short

# sharp ratio計算
def calc_sharpe_ratio(df):
    daily_ret = df.groupby('Date').apply(calc_return_per_day)
    sharpe_ratio = daily_ret.mean() / daily_ret.std()
    return sharpe_ratio

# 予測用のデータフレームと、予測結果をもとに、sharp ratioを計算する関数
def evaluator(df, pred):
    df["pred"] = pred
    df = add_rank(df)
    sharp_ratio = calc_sharpe_ratio(df)
    return sharp_ratio

# モデル

In [ ]:
fold_params = [
    ('2020-12-23', '2021-11-01', '2021-12-01'),
    ('2021-01-23', '2021-12-01', '2022-01-01'),
    ('2021-02-23', '2022-01-01', '2022-02-01'),
]

def objective(trial):

    scores = []
    models = []
    params = []

    for param in fold_params:
        ################################
        # データ準備
        ################################
        # データを train と valid　に分ける
        train = feature_df[(param[0] <= feature_df['Date']) & (feature_df['Date'] < param[1])]
        valid = feature_df[(param[1] <= feature_df['Date']) & (feature_df['Date'] < param[2])]

        X_train = train[feat_cols]
        y_train = train[label_col]
        X_valid = valid[feat_cols]
        y_valid = valid[label_col]


        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        learning_rate = trial.suggest_uniform('learning_rate', 0.1, 1.0)
        num_leaves =  trial.suggest_int("num_leaves", 5, 50)
        lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-8, 10.0)
        lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-8, 10.0)
        feature_fraction = trial.suggest_uniform('feature_fraction', 0.4, 1.0)
        bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.4, 1.0)
        bagging_freq = trial.suggest_int('bagging_freq', 1, 7)
        min_child_samples = trial.suggest_int('min_child_samples', 5, 100)

        params = {
                'task': 'train',                   # 学習
                'boosting_type': 'gbdt',           # GBDT
                'objective': 'regression',         # 回帰
                'metric': 'rmse',                  # 損失（誤差）

                'learning_rate': learning_rate,  # 学習率
                'num_leaves': num_leaves,
                'lambda_l1': lambda_l1,               
                'lambda_l2': lambda_l2 ,           
                'feature_fraction': feature_fraction,           # ランダムに抽出される列の割合
                'bagging_fraction': bagging_fraction,           # ランダムに抽出される標本の割合
                'bagging_freq': bagging_freq,                 # バギング実施頻度
                'min_child_samples': min_child_samples,           # 葉に含まれる最小データ数

                'seed': 2022,                         # シード値
                "force_col_wise":True,
                "extra_trees":True,
                }

        lgb_results = {}   

        model = lgb.train(
                        params=params,                    # ハイパーパラメータをセット
                        train_set=lgb_train,              # 訓練データを訓練用にセット
                        valid_sets=[lgb_train, lgb_valid], # 訓練データとテストデータをセット
                        valid_names=['Train', 'Valid'],    # データセットの名前をそれぞれ設定
                        num_boost_round=100,              # 計算回数
                        early_stopping_rounds=50,         # アーリーストッピング設定
                        evals_result=lgb_results,
                        verbose_eval=-1,                  # ログを最後の1つだけ表示
                        )


        pred =  model.predict(X_valid, num_iteration=model.best_iteration)
        score = evaluator(valid, pred)

    return score

opt = optuna.create_study(direction='maximize')
opt.optimize(objective, n_trials=50)

opt.best_params


In [ ]:
{'learning_rate': 0.4309272728230116,
 'num_leaves': 26,
 'lambda_l1': 0.015061607016359854,
 'lambda_l2': 0.014098154084387682,
 'feature_fraction': 0.8241286866590853,
 'bagging_fraction': 0.6249932406838286,
 'bagging_freq': 5,
 'min_child_samples': 77}

In [ ]:
# 学習
def trainer(feature_df, feat_cols, label_col, fold_params):
    scores = []
    models = []
    params = []

    for param in fold_params:
        ################################
        # データ準備
        ################################
        # データを train と valid　に分ける
        train = feature_df[(param[0] <= feature_df['Date']) & (feature_df['Date'] < param[1])]
        valid = feature_df[(param[1] <= feature_df['Date']) & (feature_df['Date'] < param[2])]

        X_train = train[feat_cols]
        y_train = train[label_col]
        X_valid = valid[feat_cols]
        y_valid = valid[label_col]
        
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        ################################
        # 学習
        ################################
        params = {
            'task': 'train',                   # 学習
            'boosting_type': 'gbdt',           # GBDT
            'objective': 'regression',         # 回帰
            'metric': 'rmse',                  # 損失（誤差）
            
            'learning_rate': 0.4309272728230116,
            'num_leaves': 26,
            'lambda_l1': 0.015061607016359854,
            'lambda_l2': 0.014098154084387682,
            'feature_fraction': 0.8241286866590853,
            'bagging_fraction': 0.6249932406838286,
            'bagging_freq': 5,
            'min_child_samples': 77,
#             'learning_rate': 0.001,             # 学習率
#             'lambda_l1': 0.5,                  # L1正則化項の係数
#             'lambda_l2': 0.5,                  # L2正則化項の係数
#             'num_leaves': 15,                  # 最大葉枚数
#             'feature_fraction': 0.5,           # ランダムに抽出される列の割合
#             'bagging_fraction': 0.5,           # ランダムに抽出される標本の割合
#             'bagging_freq': 5,                 # バギング実施頻度
#             'min_child_samples': 10,           # 葉に含まれる最小データ数
            'seed': 2022,                         # シード値
            "force_col_wise":True,
            "extra_trees":True,
        } 
 
        lgb_results = {}             
    
        model = lgb.train( 
            params,                            # ハイパーパラメータ
            lgb_train,                         # 訓練データ
            valid_sets=[lgb_train, lgb_valid], # 検証データ
            valid_names=['Train', 'Valid'],    # データセット名前
            num_boost_round=2000,              # 計算回数
            early_stopping_rounds=100,         # 計算打ち切り設定
            evals_result=lgb_results,          # 学習の履歴
            verbose_eval=-1,                   # 学習過程の表示サイクル
        )  

        ################################
        # 結果描画
        ################################
        
        fig = plt.figure(figsize=(10, 4))

        # loss
        plt.subplot(1,2,1)
        loss_train = lgb_results['Train']['rmse']
        loss_test = lgb_results['Valid']['rmse']   
        plt.xlabel('Iteration')
        plt.ylabel('logloss')
        plt.plot(loss_train, label='train loss')
        plt.plot(loss_test, label='valid loss')
        plt.legend()

        # feature importance
        plt.subplot(1,2,2)
        importance = pd.DataFrame({'feature':feat_cols, 'importance':model.feature_importance()})
        sns.barplot(x = 'importance', y = 'feature', data = importance.sort_values('importance', ascending=False))

        plt.tight_layout()#文字が重らないように
        plt.show()

        ################################
        # 評価
        ################################
        pred =  model.predict(X_valid, num_iteration=model.best_iteration)
        # 評価
        score = evaluator(valid, pred)

        scores.append(score)
        models.append(model)

    print("CV_SCORES:", scores)
    print("CV_SCORE:", np.mean(scores))
    
    return models

# 結果出力
# 2020-12-23よりも前のデータは証券コードが2000個すべて揃っていないため、これ以降のデータのみを使う。
# (学習用データの開始日、学習用データの終了日＝検証用データの開始日、検証用データの終了日)
fold_params = [
    ('2020-12-23', '2021-11-01', '2021-12-01'),
    ('2021-01-23', '2021-12-01', '2022-01-01'),
    ('2021-02-23', '2022-01-01', '2022-02-01'),
]
models = trainer(feature_df, feat_cols, label_col, fold_params)

In [ ]:
# テストデータで最終検証
def predictor(feature_df, feat_cols, models, is_train=True):
    X = feature_df[feat_cols]
    
    preds = list(map(lambda model: model.predict(X, num_iteration=model.best_iteration), models))
    
    # スコアは学習時のみ計算
    if is_train:
        scores = list(map(lambda pred: evaluator(feature_df, pred), preds))
        print("SCORES:", scores)

    # 推論結果をバギング
    pred = np.array(preds).mean(axis=0)

    # スコアは学習時のみ計算
    if is_train:
        score = evaluator(feature_df, pred)
        print("SCORE:", score)
    
    return pred

# 試験用データは学習用にも検証用にも使用していないものを使う
test_df = feature_df[('2022-02-01' <= feature_df['Date'])].copy()
pred = predictor(test_df, feat_cols, models)

In [ ]:
# 時系列APIのロード
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

# supplemental filesを履歴データの初期状態としてセットアップ
past_df = supplemental_df.copy()

# 日次で推論・登録
for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(iter_test):
    current_date = prices["Date"].iloc[0]

    if i == 0:
        # リークを防止するため、時系列APIから受け取ったデータより未来のデータを削除
        past_df = past_df[past_df["Date"] < current_date]

    # リソース確保のため古い履歴を削除
    threshold = (pd.Timestamp(current_date) - pd.offsets.BDay(80)).strftime("%Y-%m-%d")
    past_df = past_df[past_df["Date"] >= threshold]
    
    # 時系列APIから受け取ったデータを履歴データに統合
    base_df = collector(prices, options, financials, trades, secondary_prices, stock_list)
    past_df = pd.concat([past_df, base_df]).reset_index(drop=True)

    # 特徴量エンジニアリング
    feature_df, feat_cols, label_col = preprocessor(past_df)

    # 予測対象レコードだけを抽出
    feature_df = feature_df[feature_df['Date'] == current_date]

    # 推論
    feature_df["pred"] = predictor(feature_df, feat_cols, models, False)

    # 推論結果からRANKを導出し、提出データに反映
    feature_df = add_rank(feature_df)
    feature_map = feature_df.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)

    # 結果を登録
    env.predict(sample_prediction)